In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [2]:
def months_diff(df, start_date, end_date):
    month_diff = end_date-df[start_date].dt.to_period('M').astype(int)
    if df[start_date].isnull().sum()>0:
        month_diff=month_diff.mask(df[start_date].isnull(), np.nan)
    return month_diff

In [3]:
def segment_conversion(df, group_var, response_var, levels):
    if levels >0:
        agg= df.groupby([group_var])[response_var].agg(['count', 'mean']).sort_values(by='count', ascending=False)[:levels]
    else:
        agg= df.groupby([group_var])[response_var].agg(['count', 'mean']).sort_values(by='count', ascending=False)
    return agg.reset_index()

In [4]:
def data_pipeline(df, dependent):
    """
    Preprocessing pipeline 1
    exclude dependent variables
    """
    X, y = df.loc[:, df.columns!=dependent], df[[dependent]]
    X = drop_missing_columns(X, 0.5)
    X = drop_datetime(X)
    X = drop_services(X)
    X = drop_sparse_variable(X, 0.1)
    X = categorize_numerics(X, 5)
    #X = cap_scale(X, 0.9)
    X = one_hot_encode(X,y, 0.25, 0.25, 0.75)
    
    return X, y

In [5]:
def drop_missing_columns(df, threshold):
    """Drop columns with missing values more than threshold"""
    drop_columns=[]
    for c in list(df.columns):
        if df[c].count()<len(df)*threshold:
            drop_columns.append(c)
    return df.drop(drop_columns, axis=1)

In [6]:
def drop_datetime(df):
    dt_stamps=df.filter(regex='date$|time$|months$', axis=1).columns
    return df.drop(dt_stamps, axis=1)

In [7]:
def drop_services(df):
    dt_stamps=df.filter(regex='services$|count$|active$', axis=1).columns
    return df.drop(dt_stamps, axis=1)

In [8]:
def categorize_numerics(df, threshold):
    """
    Convert numerical variables with discrete unique values
    """
    numerics=list(df.select_dtypes(include=['float64', 'int64']).columns)
    for n in numerics:
        if df[n].nunique()<threshold:
            df[n]=df[n].astype(str)          
    return df  

In [9]:
def drop_high_zeros(df, threshold):
    """
    Drop numerical values with high percentage of zeros
    """
    drop_list=[]
    numerics=list(df.select_dtypes(include=['float64', 'int64']).columns)
    for n in numerics:
        if df[n].eq(0).sum()> threshold*len(df):
            drop_list.append(n)
            
    if len(drop_list)>0:
        df=df.drop(drop_list, axis=1)
    return df

In [10]:
def drop_sparse_variable(df, threshold):
    """
    Drop the variables if it has too many unique values
    No need to use it if already doing the one_hot_encode
    """
    categoricals= list(df.select_dtypes(include='object').columns)
    sparses=[]
    for c in categoricals:
        if df[c].nunique()>threshold*len(df):
            sparses.append(c)
    if len(sparses)>0:
        df=df.drop(sparses, axis=1)
    return df

In [11]:
def cap_scale(df, threshold):
    """
    Replace the values with the the threshold for those above threshold
    MinMaxscaler to tranform the values into range(0,1)
    """
    numerics=list(df.select_dtypes(include=['float64', 'int64']).columns)
    scaler=MinMaxScaler()
    for n in numerics:
        if 'month' not in n:
            if df[n].max()>1:
                cap=df[n].quantile(threshold)
                df[n]=df[n].where(df[n]<=cap, cap)
                df.loc[:, n]=scaler.fit_transform(df[[n]])
        else:
            df.loc[:, n]=scaler.fit_transform(df[[n]]) 
    return df

In [12]:
def one_hot_encode(df, target, count_threshold, lower_conversion, upper_conversion):
    """
    Only create binary variables for meaningful values out of categorical variables
    """
    categoricals= list(df.select_dtypes(include='object').columns)
    df_combined=pd.concat([df, target], axis=1)
    for c in categoricals:
        sum_table=segment_conversion(df_combined, c, 'marketing_campaign_response', -1)
        sum_table=sum_table[(sum_table['count']>count_threshold*len(df))&
                            ((sum_table['mean']<lower_conversion)| 
                             (sum_table['mean']>upper_conversion))]
        if len(sum_table)>0:
            dummy_values=sum_table[c].values
            if len(df[c].unique())==2 and len(sum_table)==2:
                df=pd.get_dummies(df, columns=[c], prefix=[c], drop_first=True)
            else:
                df[c]=df[c].where(df[c].isin(dummy_values), 'other')
                df=pd.get_dummies(df, columns=[c], prefix=[c], drop_first=False)
                df=df.drop([c+'_other'], axis=1)
        else:
            df=df.drop(c, axis=1)
    return df